In [2]:
import copy
import numpy as np
import math

In [3]:
from copy import deepcopy
from copy import copy
import numpy as np


NUM_ROWS = 6
NUM_COLUMNS = 6
#TWO_P = [5, 5, 4, 3, 2]
TWO_P = [2,2]

class HalmaGame:
    def __init__(self, board_new=None):
        self.goal_board = np.zeros([NUM_ROWS, NUM_COLUMNS])
        for i in range(len(TWO_P)):
                self.goal_board[i, :TWO_P[i]] = 2
                self.goal_board[-i - 1, -TWO_P[i]:] = 1
        self.board = board_new
        if np.array_equal(board_new, None):
            self.board = np.zeros([NUM_ROWS, NUM_COLUMNS])
            for i in range(len(TWO_P)):
                self.board[i, :TWO_P[i]] = 1
                self.board[-i - 1, -TWO_P[i]:] = 2

        self.current_player = 1
        self.winner = -1

        self.child_boards = []
        self.parent = None
        self.parent_operator = None
        self.jump_flag = 0
        self.new_idx = None

    def __eq__(self, other):
        return np.array_equal(self.board, other.board)

    def __hash__(self):
        return hash(str([item for sublist in self.board for item in sublist]))

    def check_winner(self, game):
        goal_player_1 = np.where(game.goal_board == 1)
        goal_player_2 = np.where(game.goal_board == 2)
        board_index_1 = np.where(game.board == 1)
        board_index_2 = np.where(game.board == 2)
        
        if np.array_equal(goal_player_1, board_index_1):
            self.winner = 1
        elif np.array_equal(goal_player_2, board_index_2):
            self.winner = 2
        else:
            self.winner = -1
        
    def move(self, move):
        state_copy = deepcopy(self)
        state_copy.board = move.board
        state_copy.check_winner(state_copy)
        state_copy.current_player = state_copy.switch_player(state_copy)
        return state_copy

    def set_player(self, p):
        self.player = p
        
    def set_board(self, b):
        self.board = b
        
    def switch_player(self, state):
        if state.current_player == 1:
            return 2
        if state.current_player == 2:
            return 1

        
    def setSimpleMove(self, kid, parent_op, p, switch_p=0, jump_f = 0, newindex = None):
        kid.parent = p
        kid.parent_operator = parent_op
        if switch_p:
            kid.current_player = self.switch_player(switch_p)
        kid.jump_flag = jump_f
        kid.new_idx = newindex
        
      
    def moves_check(self, player):
        visited = set()
        visited.add(self)
        queue = [self]
        all_kids = set()
        jump_flag = 0
        pieces = np.array(np.where(self.board == player)).T

        #for idx in pieces:
        for i in range(len(pieces)):
            idx = deepcopy(pieces[i])
            # up
            if idx[0] - 1 >= 0:
                if not self.board[idx[0] - 1, idx[1]]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1]] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up", self)
                    all_kids.add(child)

            # up_right
            if idx[0] - 1 >= 0 and idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0] - 1, idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up_right", self)
                    all_kids.add(child)

            # right
            if idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0], idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0], idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "right", self)
                    all_kids.add(child)

            # down_right
            if idx[0] + 1 < NUM_ROWS and idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0] + 1, idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down_right", self)
                    all_kids.add(child)

            # down
            if idx[0] + 1 < NUM_ROWS:
                if not self.board[idx[0] + 1, idx[1]]:
                    #child = deepcopy(self)
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1]] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down", self)
                    all_kids.add(child)

            # down left
            if idx[0] + 1 < NUM_ROWS and idx[1] - 1 >= 0:
                if not self.board[idx[0] + 1, idx[1] - 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down_left", self)
                    all_kids.add(child)

            # left
            if idx[1] - 1 >= 0:
                if not self.board[idx[0], idx[1] - 1]:
                    #child = deepcopy(self)
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0], idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "left", self)
                    all_kids.add(child)

            # up_left
            if idx[0] - 1 >= 0 and idx[1] - 1 >= 0:
                if not self.board[idx[0] - 1, idx[1] - 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up_left", self)
                    all_kids.add(child)

            while len(queue):
                node = queue.pop(0)
                if node.jump_flag:
                    idx = node.new_idx
                    
                if idx[0] - 2 >= 0:
                    if node.board[idx[0] - 1, idx[1]] and not node.board[idx[0] - 2, idx[1]]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] - 2, idx[1]] = player
                        child = HalmaGame(new_board)                      
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up", node, jump_f = 1, newindex = [idx[0] - 2, idx[1]])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0


                # jump up right
                if idx[0] - 2 >= 0 and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] - 1, idx[1] + 1] and not node.board[idx[0] - 2, idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] - 2, idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up_right", node, jump_f = 1, newindex = [idx[0] - 2, idx[1]+2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                            
                            
                # jump right
                if idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0], idx[1] + 1] and not node.board[idx[0], idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0], idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_right", node, jump_f = 1, newindex = [idx[0], idx[1] + 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down right
                if idx[0] + 2 < NUM_ROWS and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] + 1, idx[1] + 1] and not node.board[idx[0] + 2, idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down_right", node, jump_f = 1, newindex = [idx[0] + 2, idx[1] + 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down
                if idx[0] + 2 < NUM_ROWS:
                    if node.board[idx[0] + 1, idx[1]] and not node.board[idx[0] + 2, idx[1]]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1]] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down", node, jump_f = 1, newindex = [idx[0] + 2, idx[1]])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down left
                if idx[0] + 2 < NUM_ROWS and idx[1] - 2 >= 0:
                    if node.board[idx[0] + 1, idx[1] - 1] and not node.board[idx[0] + 2, idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down_left", node, jump_f = 1, newindex = [idx[0] + 2, idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
                # jump left
                if idx[1] - 2 >= 0:
                    if node.board[idx[0], idx[1] - 1] and not node.board[idx[0], idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0], idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_left", node, jump_f = 1, newindex = [idx[0], idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
                # jump up left
                if idx[1] - 2 >= 0 and idx[0] -2 >= 0:
                    if node.board[idx[0]-1, idx[1] - 1] and not node.board[idx[0]-2, idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0]-2, idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up_left", node, jump_f = 1, newindex = [idx[0]-2, idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
            queue=[self]
        self.child_boards = list(all_kids)
        return tuple(self.child_boards)

In [4]:
def minimax(state, depth, maximizing, player, alpha, beta, evaluation_function):
    if depth == 0 or state.winner != -1:
        if evaluation_function == evaluation_function_1:
            return evaluation_function(state) * (-1 if player == 1 else 1)
        elif evaluation_function == evaluation_function_2:
            return evaluation_function(state)

    if maximizing:
        max_val = float('-inf')
        possible_moves = state.moves_check(player)
        for move in possible_moves:
            new_state = state.move(move)
            if new_state.winner != -1:
                return evaluation_function(new_state)
            evaluation = minimax(new_state, depth - 1, False, player, alpha, beta, evaluation_function)
            max_val = max(max_val, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_val

    else:
        min_val = float('inf')
        possible_moves = state.moves_check(player)
        for move in possible_moves:
            new_state = state.move(move)
            if new_state.winner != -1:
                return evaluation_function(new_state)
            evaluation = minimax(new_state, depth - 1, True, player, alpha, beta, evaluation_function)
            min_val = min(min_val, evaluation)
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_val

def execute_move(evaluate_func, depth):
    def execute_minimax_move_aux(game):
        best_move = None
        alpha = float('-inf')
        beta = float('inf')
        if evaluate_func == evaluation_function_1:
            best_eval = float('-inf')
            for move in game.state.moves_check(game.state.current_player):
                new_state = game.state.move(move)
                new_state_eval = minimax(new_state, depth - 1, False, game.state.current_player, alpha, beta, evaluate_func)
                # When new_state_eval is None, it means it's a finishing move
                if new_state_eval is not None:
                    if new_state_eval > best_eval:
                        best_move = new_state
                        best_eval = new_state_eval
                else:
                    best_move = new_state
            game.state = best_move
        elif evaluate_func == evaluation_function_2:
            best_eval = float('inf')
            for move in game.state.moves_check(game.state.current_player):
                new_state = game.state.move(move)
                new_state_eval = minimax(new_state, depth - 1, True, game.state.current_player, alpha, beta, evaluate_func)
                # When new_state_eval is None, it means it's a finishing move
                if new_state_eval is not None:
                    if new_state_eval < best_eval:
                        best_move = new_state
                        best_eval = new_state_eval
                else:
                    best_move = new_state
            game.state = best_move
    return execute_minimax_move_aux


In [5]:
import random

def execute_random_move(game):
    
    move = random.choice(game.state.moves_check(game.state.current_player))
    game.state = game.state.move(move)
    

In [6]:
def execute_human_move(game, consecutive=False, piece_index=[]):
    
    if consecutive:
        pieces = piece_index
    else:
        piece = np.array(list(input('Please select the index of the piece you want to move: ')))
        if piece.shape != (3,):
            piece = np.array(list(input('Please insert a correct index input: ')))
    
        pieces = []
        pieces.append(int(piece[0]))
        pieces.append(int(piece[2]))

    move = input('Please select the name of the move you want to make: ')

    # up
    if pieces[0] - 1 >= 0 and move == 'up':
        if not game.state.board[pieces[0] - 1, pieces[1]]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] - 1, pieces[1]] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # up_right
    if pieces[0] - 1 >= 0 and pieces[1] + 1 < NUM_COLUMNS and move == 'up_right':
        if not game.state.board[pieces[0] - 1, pieces[1] + 1]:
            game.state.board[idx[0], idx[1]] = 0
            game.state.board[idx[0] - 1, idx[1] + 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # right
    if pieces[1] + 1 < NUM_COLUMNS and move == 'right':
        if not game.state.board[pieces[0], pieces[1] + 1]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0], pieces[1] + 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # down_right
    if pieces[0] + 1 < NUM_ROWS and pieces[1] + 1 < NUM_COLUMNS and move == 'down_right':
        if not game.state.board[pieces[0] + 1, pieces[1] + 1]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 1, pieces[1] + 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # down
    if pieces[0] + 1 < NUM_ROWS and move == 'down':
        if not game.state.board[pieces[0] + 1, pieces[1]]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 1, pieces[1]] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # down left
    if pieces[0] + 1 < NUM_ROWS and pieces[1] - 1 >= 0 and move == 'down_left':
        if not game.state.board[pieces[0] + 1, pieces[1] - 1]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 1, pieces[1] - 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # left
    if pieces[1] - 1 >= 0 and move == 'left':
        if not game.state.board[pieces[0], pieces[1] - 1]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0], pieces[1] - 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)

    # up_left
    if pieces[0] - 1 >= 0 and pieces[1] - 1 >= 0 and move == 'up_left':
        if not game.state.board[pieces[0] - 1, pieces[1] - 1]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] - 1, pieces[1] - 1] = game.state.current_player
            game.state.current_player = game.state.switch_player(game.state)


    if pieces[0] - 2 >= 0 and move == 'jump_up':
        if game.state.board[pieces[0] - 1, pieces[1]] and not game.state.board[pieces[0] - 2, pieces[1]]:
            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] - 2, pieces[1]] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] - 2, pieces[1]]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)


    # jump up right
    if pieces[0] - 2 >= 0 and pieces[1] + 2 < NUM_COLUMNS and move == 'jump_up_right':
        if game.state.board[pieces[0] - 1, pieces[1] + 1] and not game.state.board[pieces[0] - 2, pieces[1] + 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] - 2, pieces[1] + 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] - 2, pieces[1] + 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump right
    if pieces[1] + 2 < NUM_COLUMNS and move == 'jump_right':
        if game.state.board[pieces[0], pieces[1] + 1] and not game.state.board[pieces[0], pieces[1] + 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0], pieces[1] + 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0], pieces[1] + 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump down right
    if pieces[0] + 2 < NUM_ROWS and pieces[1] + 2 < NUM_COLUMNS and move == 'jump_down_right':
        if game.state.board[pieces[0] + 1, pieces[1] + 1] and not game.state.board[pieces[0] + 2, pieces[1] + 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 2, pieces[1] + 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] + 2, pieces[1] + 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump down
    if pieces[0] + 2 < NUM_ROWS and move == 'jump_down':
        if game.state.board[pieces[0] + 1, pieces[1]] and not game.state.board[pieces[0] + 2, pieces[1]]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 2, pieces[1]] = game.state.current_player
            
############################ Checking for indexes larger than the board size ###########################################################
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] + 2, pieces[1]]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump down left
    if pieces[0] + 2 < NUM_ROWS and pieces[1] - 2 >= 0 and move == 'jump_down_left':
        if game.state.board[pieces[0] + 1, pieces[1] - 1] and not game.state.board[pieces[0] + 2, pieces[1] - 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] + 2, pieces[1] - 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] + 2, pieces[1] - 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump left
    if pieces[1] - 2 >= 0 and move == 'jump_left':
        if game.state.board[pieces[0], pieces[1] - 1] and not game.state.board[pieces[0], pieces[1] - 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0], pieces[1] - 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0], pieces[1] - 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)

    # jump up left
    if pieces[1] - 2 >= 0 and pieces[0] -2 >= 0 and move == 'jump_up_left':
        if game.state.board[pieces[0]-1, pieces[1] - 1] and not game.state.board[pieces[0]-2, pieces[1] - 2]:

            game.state.board[pieces[0], pieces[1]] = 0
            game.state.board[pieces[0] - 2, pieces[1] - 2] = game.state.current_player
            if game.state.board[pieces[0] - 1, pieces[1]] or game.state.board[pieces[0] - 1, pieces[1] + 1] or game.state.board[pieces[0] , pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1] + 1] or game.state.board[pieces[0] + 1, pieces[1]] or game.state.board[pieces[0] + 1, pieces[1] - 1] or game.state.board[pieces[0], pieces[1] - 1] or game.state.board[pieces[0] - 1, pieces[1] - 1]:
                print('Consecutive jump is possible')
                option = input("Do you wish to jump again? If so type 'yes', if not type 'no': " )
                if option == 'no':
                    game.state.current_player = game.state.switch_player(game.state)
                elif option == 'yes':
                    pieces = [pieces[0] - 2, pieces[1] - 2]
                    execute_human_move(game, True, pieces)
                else:
                    execute_human_move(game, True, pieces)
                    print('Not an option.')
            #game.state.current_player = game.state.switch_player(game.state)



In [7]:
import time

class ConnectHalmaGame:
    
    def __init__(self, player_1_ai, player_2_ai):
        
        self.player_1_ai = player_1_ai 
        self.player_2_ai = player_2_ai 
        self.ngame = None
        
    def start(self, log_moves = False):
        self.state = HalmaGame()
        if self.ngame%2:
            self.state.current_player = 2
        while True:
            if self.state.current_player == 1:
                self.player_1_ai(self)
            else:
                self.player_2_ai(self)
            
            if log_moves:
                print(game.state.board)
            
            if self.state.winner != -1:
                break
                
        if self.state.winner == 0:
            print("End of game! Draw!")
        else:
            print(f"End of game! Player {self.state.winner} wins!")
            
    def run_n_matches(self, n, max_time = 3600, log_moves = False):
        start_time = time.time()
        
        results = [0, 0, 0]
        
        while n > 0 and time.time() - start_time < max_time:
            n -= 1
            self.ngame = n
            game.start(log_moves)
            results[game.state.winner] += 1
            
        print("\n=== Elapsed time: %s seconds ===" % (int(time.time() - start_time)))
        print(f"  Player 1: {results[1]} victories")
        print(f"  Player 2: {results[2]} victories")
        print(f"  Draws: {results[0]} ")
        print("===============================")

In [8]:
# Define evaluation functions

def evaluation_function_1(game):
    board_goal = game.goal_board
    board_state = game.board
    indexes_target_2 = np.where(board_goal == 2)
    indexes_in_2 = np.where(board_goal == 1)
    indexes_state_2 = np.where(board_state == 2)
    indexes_state_1 = np.where(board_state == 1)
    index_list_inital_1 = []
    index_list_target_1 = []
    index_list_inital_2 = []
    index_list_target_2 = []
    index_list_2 = []
    index_list_1 = []
    for i in range(len(indexes_in_2[0])):
        index_in_2 = [indexes_in_2[0][i], indexes_in_2[1][i]]
        index_list_inital_2.append(index_in_2)
        index_tg_2 = [indexes_target_2[0][i], indexes_target_2[1][i]]
        index_list_target_2.append(index_tg_2)
        index_2 = [indexes_state_2[0][i], indexes_state_2[1][i]]
        index_list_2.append(index_2)
        index_1 = [indexes_state_1[0][i], indexes_state_1[1][i]]
        index_list_1.append(index_1)
        
    index_list_inital_1 = index_list_target_2
    index_list_target_1 = index_list_inital_2

    max_dist_1 = []
    max_dist_2 = []
    #if index_list_inital_1 == index_list_target_1:
    #    game.winner = 1
    #    return 
    #if index_list_inital_2 == index_list_target_2:
    #    game.winner = 2
    #    return
    
    for i in range(len(indexes_in_2[0])):
        dist_it_1 = []
        dist_it_2 = []
        
        if index_list_1[i] not in index_list_target_1:
            for j in range(len(indexes_in_2[0])):
                    dist_it_1.append(math.dist(index_list_1[i], index_list_target_1[j]))
                    
            max_dist_1.append(min(dist_it_1))
        else:
            max_dist_1.append(0)
            
        if index_list_2[i] not in index_list_target_2:
            for j in range(len(indexes_in_2[0])):
                    dist_it_2.append(math.dist(index_list_2[i], index_list_target_2[j]))
                
            max_dist_2.append(min(dist_it_2))
        else:
            max_dist_2.append(0)
        
    distances_1 = round(sum(max_dist_1), 5)
    distances_2 = round(sum(max_dist_2), 5)
    diff = distances_1 - distances_2
    return diff

def evaluation_function_2(game):
    board_goal = game.goal_board
    board_state = game.board
    player = (1 if game.current_player == 1 else 2)
    indexes_target = np.where(board_goal == player)
    indexes_state = np.where(board_state == player)
    index_list = []
    index_list_target = []
    
    
    for i in range(len(indexes_target[0])):
        index = [indexes_state[0][i], indexes_state[1][i]]
        index_list.append(index)
        index_tg = [indexes_target[0][i], indexes_target[1][i]]
        index_list_target.append(index_tg)

    max_dist = []
    if index_list == index_list_target:
        game.winner = game.current_player
        return 
    
    for i in range(len(indexes_target[0])):
        dist_it = []
        if index_list[i] not in index_list_target:
            for j in range(len(indexes_target[0])):
                dist_it.append(math.dist(index_list[i], index_list_target[j]))
                
        if dist_it == []:
            max_dist.append(0)
        else:
            max_dist.append(min(dist_it))
    
    distances = round(sum(max_dist), 5)
    return distances

def evaluation_function_3(state):
    e_pos1 = S_Positions[1]
    e_pos2 = S_Positions[0]
    e_occ1 = ~(state.player_positions[0]== e_pos1).all(axis = 1)
    e_occ2 = ~(state.player_positions[1]== e_pos2).all(axis = 1)
    d1 = np.linalg.norm([min(np.linalg.norm(i-e_pos1, axis = 1)[e_occ1]) for i in state.player_positions[0]])
    d2 = np.linalg.norm([min(np.linalg.norm(i-e_pos2, axis = 1)[e_occ2]) for i in state.player_positions[1]])
    return d1-d2


# Create two agents with different levels of difficulty
#agent1 = HalmaAgent(evaluation_function_1, 1)
#agent2 = HalmaAgent(evaluation_function_1, 1)
#game = ConnectHalmaGame(agent1.execute_move(game), agent2.execute_move(game))
#game = ConnectHalmaGame(execute_random_move, execute_move(evaluation_function_1, 2))
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 1), execute_move(evaluation_function_2, 2))
#print('Example of move names: "up", "down_right", "jump_up_left", etc... \n')
#print('Example of piece indexes: 1,2 ; 0,0 ; 1,1; ...\n')
#WORKS REALLY GOOOOOODDDDD -> good for presentation
#game = ConnectHalmaGame(execute_human_move, execute_move(evaluation_function_1, 3)) WORKS REALLY GOOOOOODDDDD
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 2), execute_move(evaluation_function_2, 2)) works as well - 2 wins
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_2, 2)) works as well - 1 wins
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_1, 1)) - 2 wins
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_2, 1)) - doenst end, 2 doenst move
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_1, 1)) 
#- works - 1 wins
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_1, 3)) - doenst end
#game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_2, 2))


#game = ConnectHalmaGame(execute_human_move, execute_move(evaluation_function_1, 3))
#game.run_n_matches(5, 300, False)


In [132]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 1), execute_move(evaluation_function_1, 1))
game.run_n_matches(6, 300, False)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 0 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 


In [133]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 2), execute_move(evaluation_function_1, 1))
game.run_n_matches(6, 300, False)

End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!

=== Elapsed time: 5 seconds ===
  Player 1: 0 victories
  Player 2: 6 victories
  Draws: 0 


In [134]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 2), execute_move(evaluation_function_1, 2))
game.run_n_matches(6, 300, False)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 29 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 


In [135]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_1, 2))
game.run_n_matches(6, 300, False)

End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!
End of game! Player 2 wins!
End of game! Player 1 wins!

=== Elapsed time: 83 seconds ===
  Player 1: 3 victories
  Player 2: 3 victories
  Draws: 0 


In [136]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 1), execute_move(evaluation_function_2, 2))
game.run_n_matches(6, 300, False)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 8 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 


In [137]:
game = ConnectHalmaGame(execute_move(evaluation_function_1, 3), execute_move(evaluation_function_2, 2))
game.run_n_matches(6, 300, False)

End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!
End of game! Player 1 wins!

=== Elapsed time: 84 seconds ===
  Player 1: 6 victories
  Player 2: 0 victories
  Draws: 0 


In [138]:
game = ConnectHalmaGame(execute_random_move, execute_move(evaluation_function_1, 2))
game.run_n_matches(6, 300, False)

End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!
End of game! Player 2 wins!

=== Elapsed time: 19 seconds ===
  Player 1: 0 victories
  Player 2: 6 victories
  Draws: 0 


In [14]:
print('Example of move names: "up", "down_right", "jump_up_left", etc... \n')
print('Example of piece indexes: 1,2 ; 0,0 ; 1,1; ...\n')
board = HalmaGame()
print('Inital state \n', board.__dict__)
game = ConnectHalmaGame(execute_human_move, execute_move(evaluation_function_1, 1))
game.run_n_matches(1, 300, True)

Example of move names: "up", "down_right", "jump_up_left", etc... 

Example of piece indexes: 1,2 ; 0,0 ; 1,1; ...

Inital state 
 {'goal_board': array([[2., 2., 0., 0., 0., 0.],
       [2., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1.]]), 'board': array([[1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 2.],
       [0., 0., 0., 0., 2., 2.]]), 'current_player': 1, 'winner': -1, 'child_boards': [], 'parent': None, 'parent_operator': None, 'jump_flag': 0, 'new_idx': None}


KeyboardInterrupt: Interrupted by user